### Topic Modeling for Twitter

In this notebook I will apply topic modeling on the twitter data I cleaned in a separate notebook.
I will use LDA algorithm to model topic, 
Here are some usefuls ressources I used to learn the intuition behind topic modeling :
    [AV introduction to topic Modeling](https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/) 
    [LDA Video](https://www.youtube.com/watch?v=3mHy4OSyRf0) 
    [Topic Modeling With Gesim](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/)
For the first start I will use pyhon sklearn library and gensim library

In [23]:
# uncoment the next line to install gensim
# !pip install pyLDAvis

In [26]:
import pandas as pd
import gensim as gs
import numpy as np
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from pprint import pprint
from matplotlib import pyplot as plt
import pyLDAvis

In [2]:
cleanned_tweets = pd.read_csv('../data/cleanned_tweets_04-06-2019-23-07.csv', index_col='Unnamed: 0')

In [3]:
cleanned_tweets.dropna(inplace=True)

In [4]:
tweets_array = cleanned_tweets.get('cleanned_tweet').values

In [5]:
tweets_array = [tweet.split(' ') for tweet in tweets_array]

In [6]:
# Create Dictionary

id2word = corpora.Dictionary(tweets_array)

In [7]:
# Create Corpus
texts = tweets_array

In [8]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]


In [11]:
lda_model = gs.models.ldamodel.LdaModel(corpus=corpus,
                                        id2word=id2word,
                                        num_topics=5, 
                                        random_state=100,
                                        update_every=1,
                                        chunksize=100,
                                        passes=100,
                                        alpha='auto',
                                        per_word_topics=True)

In [27]:
topic_term_matrix = lda_model.get_topics()

In [32]:
topic_term_matrix[0]

array([5.6825645e-02, 5.4356784e-02, 4.9937952e-02, ..., 4.8323946e-05,
       1.0451641e-03, 1.0451641e-03], dtype=float32)

In [12]:
for label, words in lda_model.print_topics():
    print('==================')
    print("topic label ", label, ' =>word ',  words)

topic label  0  =>word  0.066*"kinshasa" + 0.065*"juin" + 0.062*"retour" + 0.057*"annonce" + 0.054*"cher" + 0.053*"dimanche" + 0.050*"joie" + 0.050*"compatriote" + 0.050*"enthousiasme" + 0.018*"bemba"
topic label  1  =>word  0.012*"nouveau" + 0.009*"ebola" + 0.008*"má" + 0.008*"jeanpierre" + 0.008*"passer" + 0.008*"argent" + 0.007*"créer" + 0.007*"affaire" + 0.007*"an" + 0.007*"journaliste"
topic label  2  =>word  0.016*"président" + 0.011*"vouloir" + 0.011*"semaine" + 0.010*"mwilambwe" + 0.009*"national" + 0.008*"gouvernement" + 0.008*"Monsieur" + 0.008*"paul" + 0.007*"politique" + 0.007*"leader"
topic label  3  =>word  0.028*"ebola" + 0.021*"outbreak" + 0.016*"kabila" + 0.015*"bureau" + 0.015*"demande" + 0.014*"assemblée" + 0.014*"provincial" + 0.012*"kinshasa" + 0.010*"ngobila" + 0.010*"révoquer"
topic label  4  =>word  0.037*"tshisekedi" + 0.013*"félix" + 0.012*"kinshaser" + 0.010*"fidèle" + 0.009*"hommage" + 0.009*"martyr" + 0.008*"président" + 0.008*"chebeya" + 0.008*"assassinat"

From the first impression the model is not that accurate , there are some with word which has no sense, the only topic I can identify from those tweet are the holy day on thurday due to Tshisekedi death .

From now , if we remove mentions and english word and article we can have better accuracy, we need to improve the model
on that side

The step to improve the model will be to remove mention, remove english word, and some useless word.

In [13]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -7.109698002056578


In [14]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4456736448363941


In [36]:
import pyLDAvis.gensim

In [47]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/espoir-mur/Projects/Personnal/twitter-analysis/.venv3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.375543 -0.058947       1        1  15.270259
5      0.015817  0.263464       2        1  13.837785
9     -0.075990  0.101088       3        1  11.819820
7      0.044542 -0.003028       4        1  11.108641
4     -0.084687 -0.222597       5        1   9.813797
3     -0.052798 -0.020694       6        1   9.083137
8     -0.038210  0.047944       7        1   8.922032
0     -0.064475 -0.011363       8        1   8.274419
1     -0.078631 -0.079986       9        1   5.986370
2     -0.041110 -0.015881      10        1   5.883742, topic_info=     Category        Freq               Term       Total  loglift  logprob
9     Default  225.000000             retour  225.000000  30.0000  30.0000
1     Default  206.000000            annonce  206.000000  29.0000  29.0000
0     Default  201.000000           @bembajp  201.000000  28.0000  28.0000
2     Default  194.000000               cher  194.000000  27.0000  27.0000
4     Default  191.000000           dimanche  191.000000  26.0000  26.0000
7     Default  233.000000               juin  233.000000  25.0000  25.0000
3     Default  181.000000        compatriote  181.000000  24.0000  24.0000
5     Default  181.000000       enthousiasme  181.000000  23.0000  23.0000
6     Default  181.000000               joie  181.000000  22.0000  22.0000
8     Default  275.000000           kinshasa  275.000000  21.0000  21.0000
202   Default  159.000000         tshisekedi  159.000000  20.0000  20.0000
453   Default  108.000000                 de  108.000000  19.0000  19.0000
213   Default  113.000000              ebola  113.000000  18.0000  18.0000
209   Default  183.000000                     183.000000  17.0000  17.0000
59    Default  186.000000                 le  186.000000  16.0000  16.0000
181   Default   82.000000          président   82.000000  15.0000  15.0000
367   Default   61.000000   @fabienkalalacim   61.000000  14.0000  14.0000
340   Default   61.000000              bemba   61.000000  13.0000  13.0000
218   Default   59.000000           outbreak   59.000000  12.0000  12.0000
19    Default   57.000000               être   57.000000  11.0000  11.0000
140   Default   44.000000             bureau   44.000000  10.0000  10.0000
27    Default   59.000000          kinshaser   59.000000   9.0000   9.0000
40    Default   44.000000                 di   44.000000   8.0000   8.0000
177   Default   49.000000    @presidence_rdc   49.000000   7.0000   7.0000
51    Default   45.000000        @politicocd   45.000000   6.0000   6.0000
915   Default   38.000000               etwm   38.000000   5.0000   5.0000
491   Default   38.000000              heure   38.000000   4.0000   4.0000
42    Default   39.000000                  e   39.000000   3.0000   3.0000
466   Default   45.000000                 un   45.000000   2.0000   2.0000
206   Default   29.000000             martyr   29.000000   1.0000   1.0000
...       ...         ...                ...         ...      ...      ...
137   Topic10   12.573106                sec   13.304515   2.7764  -4.5312
198   Topic10   11.997460             rendre   12.729016   2.7738  -4.5781
2337  Topic10   10.119486            message   10.851104   2.7632  -4.7483
135   Topic10    9.883599               port   10.615082   2.7616  -4.7719
134   Topic10    8.993386       kasumbaleser    9.724793   2.7548  -4.8663
131   Topic10    8.993386              appui    9.724793   2.7548  -4.8663
130   Topic10    8.993386           apporter    9.724793   2.7548  -4.8663
132   Topic10    8.993386                bad    9.724793   2.7548  -4.8663
133   Topic10    8.993386          financier    9.724793   2.7548  -4.8663
467   Topic10    7.514918                  è    8.246663   2.7401  -5.0459
857   Topic10    7.381611                 lo    8.113167   2.7385  -5.0638
3597  Topic10    7.088115           autorité    7.819553   2.7348  -5.1044
3387  Top

In [50]:
lda_model.get_topic_terms(1,  topn=10)

[(556, 0.017516058),
 (177, 0.017165974),
 (628, 0.016905999),
 (424, 0.016500577),
 (485, 0.01478237),
 (663, 0.014174746),
 (427, 0.013618593),
 (1675, 0.012578938),
 (432, 0.012284329),
 (426, 0.012159651)]

### II. trying Mallet Model

Let first download the model!!

In [45]:
#Uncomment to download the model
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip -O ../.venv3/models/mallet.zip

In [46]:
mallet_path = '../.venv3/models/mallet.zip' # update this path
ldamallet = gs.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command '../.venv3/models/mallet.zip import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /var/folders/nl/l4lf_0kn47d8xzr0ktf_k4xc0000gn/T/da0ad0_corpus.txt --output /var/folders/nl/l4lf_0kn47d8xzr0ktf_k4xc0000gn/T/da0ad0_corpus.mallet' returned non-zero exit status 2.